In [1]:
import sys
sys.path.append('../py_scripts')

In [2]:
from model_utils import create_simaese_model
import keras
import os
from model_utils import get_mobilenet_model, get_callbacks
from utils import get_train_test_data_dict, DictImageDataGenerator, preprocess_func

# define train params
LABEL_CNT = 5004
ALL_DATA_JSON = '../data/train_data.json'
BATCH_SIZE = 12
print('load done')
IMAGE_SHAPE = (384,384)

Using TensorFlow backend.


load done


In [3]:
# get generator
train_d, val_d = get_train_test_data_dict(ALL_DATA_JSON, 0.2, use_new_whale=False, duplicate_low_cls=True)

# simple check
cnt = 0
for k in val_d:
    if len(val_d[k])>0:
        cnt += 1
print('val cls cnt',cnt)
cnt = 0
for k in train_d:
    if len(train_d[k])>0:
        cnt += 1
print('train cls cnt',cnt)

val cls cnt 5004
train cls cnt 5004


In [4]:
train_ds = DictImageDataGenerator(rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  preprocessing_function=preprocess_func)
train_gen = train_ds.flow_from_dict_for_simaese(train_d, target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE)

val_ds = DictImageDataGenerator(preprocessing_function=preprocess_func)
val_gen = val_ds.flow_from_dict_for_simaese(val_d, target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE)

val_steps = val_gen.samples//BATCH_SIZE
print(val_steps)
# test
for [x1, x2], y in train_gen:
    print(x1.shape, x2.shape, y.shape)
    break
for [x1, x2], y in val_gen:
    print(x1.shape, x2.shape, y.shape)
    break

Found 14284 images belonging to 5004 classes.
Using bbox for image
Found 8852 images belonging to 5004 classes.
Using bbox for image
737
(12, 384, 384, 3) (12, 384, 384, 3) (12, 1)
(12, 384, 384, 3) (12, 384, 384, 3) (12, 1)


In [5]:
all_model, feat_model, compare_model = create_simaese_model(img_shape=IMAGE_SHAPE)
feat_model.summary()

/home/jac/.local/lib/python3.5/site-packages/keras_applications/mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_feat_input (InputLayer)  (None, 384, 384, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 385, 385, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 192, 192, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 192, 192, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 192, 192, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 192, 192, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 192, 192, 32)      128       
__________

In [6]:
compare_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cmp_1 (InputLayer)              (None, 512)          0                                            
__________________________________________________________________________________________________
cmp_2 (InputLayer)              (None, 512)          0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 512)          0           cmp_1[0][0]                      
                                                                 cmp_2[0][0]                      
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512)          0           cmp_1[0][0]                      
          

In [7]:
print(all_model.input_shape, all_model.output_shape)
all_model.summary()

[(None, 384, 384, 3), (None, 384, 384, 3)] (None, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
top_feat (Model)                (None, 512)          3753664     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
compare (Model)                 (None, 1)            128

In [8]:
# get generator and train
cb_list = get_callbacks('../data/checkpoints/mob_384_sim.h5', all_model)
adam_opt = keras.optimizers.Adam(lr=0.0003)
all_model.compile(optimizer=adam_opt,
                  loss='binary_crossentropy', 
                  metrics=['acc'])
print('compile done')

compile done


In [ ]:
# train
all_model.fit_generator(
    train_gen,
    steps_per_epoch=1000,
    epochs=100,
    verbose=1,
    callbacks=cb_list,
    validation_data=val_gen,
    validation_steps=val_steps
)

Epoch 1/100
   8/1000 [..............................] - ETA: 30:56 - loss: 0.7969 - acc: 0.5000